<a href="https://colab.research.google.com/github/YewonKan/DEVOIR/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [5]:
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import torch
import torch.nn as nn
import configparser

config = configparser.ConfigParser()
config.read("/sample_data/AASIST.conf")

class SSLModel(nn.Module):
    def __init__(self, device):
        super(SSLModel, self).__init__()

        # Load pre-trained Hugging Face model and processor
        self.processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
        self.model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)

        self.device = device
        self.out_dim = self.model.config.hidden_size  # 1024

    def extract_feat(self, input_data):
        # input_data shape: (batch_size, 1, waveform_length)
        if input_data.ndim == 3:
            input_data = input_data.squeeze(1)

        # Wav2Vec2Processor expects float values in range [-1, 1]
        input_data = input_data.cpu().numpy()
        inputs = self.processor(input_data, sampling_rate=16000, return_tensors="pt", padding=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Extract features from Hugging Face Wav2Vec2
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state  # shape: (batch, seq_len, hidden_dim)

        return embeddings
